# Ground Instances notebook

In [1]:
import numpy as np
import pandas as pd
import networkx as nx
%matplotlib inline
import matplotlib.pyplot as plt
import pickle
from tqdm import tqdm
import sys
sys.path.append("C:/Users/Gal/PycharmProjects/GroundInstances/")
from create_user_profile import *
from clean_data import *

Load data

In [2]:
project_dir = '/Users/Gal/Downloads/rquestions'
ans = pd.read_csv(project_dir + '/Answers.csv')
qus = pd.read_csv(project_dir + '/Questions.csv')
tags = pd.read_csv(project_dir + '/Tags.csv')

clean_data(project_dir + '/Answers.csv', project_dir + '/Clean_Answers.csv')
clean_data(project_dir + '/Questions.csv', project_dir + '/Clean_Questions.csv')
clean_data(project_dir + '/Tags.csv', project_dir + '/Clean_Tags.csv')

ans = pd.read_csv(project_dir + '/Clean_Answers.csv')
qus = pd.read_csv( project_dir + '/Clean_Questions.csv')
tags = pd.read_csv( project_dir + '/Clean_Tags.csv')

In [ ]:
ans.head()

In [ ]:
qus.head()

In [ ]:
tags.head()

# creating users list

In [ ]:
import create_user_profile
users = list_of_users(qus, ans)
users.create_user_graph()
users.set_node_properties()
users.set_user_scores(ans, qus)

In [ ]:
#saving the objects
user_df = users.generate_data_frame("C:\Users\Gal\Documents\Library-Science\user_dataframe.csv")
users.save_obj("C:\Users\Gal\Documents\Library-Science\user_list.pkl")

In [ ]:
#loading objects
user_df = pd.read_csv('C:\Users\Gal\Documents\Library-Science\user_dataframe.csv')
user_df.head()

In [ ]:
user_df[user_df['neg_questions'] > 0]

# Graph analysis

In [ ]:
G = list_of_users.load_obj('C:/Users/Gal/Documents/Library-Science/user_list.pkl')

In [ ]:
print len(list(nx.weakly_connected_component_subgraphs(G.graph))), G.graph.number_of_edges(), G.graph.number_of_nodes()
sub_graphs = list(nx.weakly_connected_component_subgraphs(G.graph))

In [ ]:
sorted([len(x) for x in sub_graphs])[-10:]

In [ ]:
g = [x for x in sub_graphs if len(x) == 11462][0]
nx.closeness_centrality(g)
nx.closeness_centrality(g)

In [ ]:
nx.draw(users.graph)

# Train/Test table 

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

final_df = ans.merge(user_df, left_on="OwnerUserId", right_on="id")
final_df = final_df.merge(qus, left_on="ParentId", right_on="Id", suffixes=("_Ans", "_Qus"), how="left")

final_df.IsAcceptedAnswer = final_df.IsAcceptedAnswer.astype(int)

final_df.dropna(inplace=True)
train, test = train_test_split(final_df, test_size=0.2)
len(train.columns), len(test.columns)

In [ ]:
final_df.head(40)

In [ ]:
# training - random forest
random_forest_clf = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=0)
features_for_training = ['Score_Qus', 'Score_Ans', 'closeness_centrality', 'in_degree', 'neg_answers', 'neg_questions', 'number_of_answers', 'out_degree', 'question_total_score']
random_forest_clf.fit(train[features_for_training], train['IsAcceptedAnswer'])

# training - logistic regression
logisticRegr = LogisticRegression(max_iter=1000, class_weight='balanced')
logisticRegr.fit(train[features_for_training], train['IsAcceptedAnswer'])
results = logisticRegr.predict(test[features_for_training]) - test['IsAcceptedAnswer'] 

#predicting
'''
results = random_forest_clf.predict(test[features_for_training])
results -= test['IsAcceptedAnswer'] 
features_for_training = ['Score']
logisticRegr.fit(train[features_for_training], train['IsAcceptedAnswer'])
results = logisticRegr.predict(test[features_for_training]) - test['IsAcceptedAnswer'] 
'''
np.count_nonzero(results) / float(len(results))


In [ ]:
final_df.head()

In [ ]:
np.count_nonzero(ans["IsAcceptedAnswer"]) / float(len(ans))

In [ ]:
import collections
c = collections.Counter(tags["Id"])
c

# Ans score - IsAccepted graph

In [ ]:
%matplotlib inline
F = [x for x in ans[ans['IsAcceptedAnswer'] == False]['Score'] if x > -10 or x<50]
T = [x for x in ans[ans['IsAcceptedAnswer'] == True]['Score'] if x > -10 or x<50]
bins = np.linspace(-10, 20, 30)

plt.hist(F, bins, alpha=0.5, label='Not accepted')
plt.hist(T, bins, alpha=0.5, label='Accepted', weights=[len(T)/float(len(F))]*len(T))

plt.legend(loc='upper right')
plt.xlabel('score')
plt.ylabel('# samples')

plt.show()

In [ ]:
len(ans), len(qus)

In [ ]:
T.head()

In [ ]:
ans[ans['IsAcceptedAnswer'] == False]['ParentId'].nunique(), len(ans[ans['IsAcceptedAnswer'] == False]['ParentId'])

In [3]:
import nltk
from nltk.corpus import stopwords
stopWords = set(stopwords.words('english'))
print stopWords

set([u'all', u'just', u'being', u'over', u'both', u'through', u'yourselves', u'its', u'before', u'o', u'hadn', u'herself', u'll', u'had', u'should', u'to', u'only', u'won', u'under', u'ours', u'has', u'do', u'them', u'his', u'very', u'they', u'not', u'during', u'now', u'him', u'nor', u'd', u'did', u'didn', u'this', u'she', u'each', u'further', u'where', u'few', u'because', u'doing', u'some', u'hasn', u'are', u'our', u'ourselves', u'out', u'what', u'for', u'while', u're', u'does', u'above', u'between', u'mustn', u't', u'be', u'we', u'who', u'were', u'here', u'shouldn', u'hers', u'by', u'on', u'about', u'couldn', u'of', u'against', u's', u'isn', u'or', u'own', u'into', u'yourself', u'down', u'mightn', u'wasn', u'your', u'from', u'her', u'their', u'aren', u'there', u'been', u'whom', u'too', u'wouldn', u'themselves', u'weren', u'was', u'until', u'more', u'himself', u'that', u'but', u'don', u'with', u'than', u'those', u'he', u'me', u'myself', u'ma', u'these', u'up', u'will', u'below', u'ain

In [7]:
import gensim
Lda = gensim.models.ldamodel.LdaModel
from gensim import corpora

qus_df = pd.read_csv('C:\\Users\\Gal\\Documents\\Library-Science\\clean_question_df.csv')
ans_df = pd.read_csv('C:\\Users\\Gal\\Documents\\Library-Science\\clean_answers_df.csv')

qus_df.head()
ans_df.head()


,Unnamed: 0,Unnamed: 0.1,Id,OwnerUserId,CreationDate,ParentId,Score,IsAcceptedAnswer,Body,clean_body
0,0,0,79741,3259.0,2008-09-17T03:43:22Z,79709,-1,False,<p>It's tough to say definitively without know...,tough say definitively without knowing languag...
1,1,1,79768,6043.0,2008-09-17T03:48:29Z,79709,9,False,<p>use variables in the outer function instead...,variables outer function instead global gets b...
2,2,2,79779,8002.0,2008-09-17T03:49:36Z,79709,0,False,<p>Third approach: inner function returns a re...,inner function returns reference large next st...
3,3,4,79827,14257.0,2008-09-17T03:58:26Z,79709,1,False,"<p>I'm not sure I understand the question, but...",sure understand couple create list vectors ret...
4,4,5,79893,14928.0,2008-09-17T04:11:08Z,79709,6,False,"<p>Remember your Knuth. ""Premature optimizati...",optimization root programming side effect free...


In [8]:
import NLP
q_docs = qus_df['clean_body']
a_docs = ans_df['clean_body']

q_docs = [x.split(" ") for x in q_docs if type(x)!=float]
a_docs = [x.split(" ") for x in a_docs if type(x)!=float]

q_dictionary = corpora.Dictionary(q_docs)
q_doc_term_matrix = [q_dictionary.doc2bow(doc) for doc in q_docs]

a_dictionary = corpora.Dictionary(a_docs)
a_doc_term_matrix = [a_dictionary.doc2bow(doc) for doc in a_docs]

Lda = gensim.models.ldamodel.LdaModel

print "building a model for questions"
q_ldamodel = Lda(q_doc_term_matrix, num_topics=10, id2word=dictionary, passes=50)
q_ldamodel.save("my_lda_model_qus_10_topics")

print "building a model for answers"
a_ldamodel = Lda(a_doc_term_matrix, num_topics=10, id2word=dictionary, passes=50)
a_ldamodel.save("my_lda_model_ans_10_topics")

building a model for questions
building a model for answers


In [10]:
for x in a_ldamodel.print_topics(num_topics=10, num_words=20):
    print x

(0, u'0.030*"ala" + 0.025*"search" + 0.017*"repository" + 0.014*"colleague" + 0.013*"easier" + 0.012*"indicating" + 0.011*"take" + 0.011*"type" + 0.011*"efficient" + 0.011*"part" + 0.010*"found" + 0.010*"matrix" + 0.010*"unique" + 0.009*"last" + 0.009*"engine" + 0.009*"dendrogram</a>" + 0.008*"leaving" + 0.008*"h" + 0.008*"proving" + 0.007*"tried"')
(1, u'0.053*"thus" + 0.026*"unlike" + 0.015*"come" + 0.015*"efficient" + 0.009*"practice" + 0.009*"leaving" + 0.008*"t[i]" + 0.007*"last" + 0.007*"125" + 0.007*"listed" + 0.007*"accessed" + 0.006*"example</a>" + 0.006*"hacked" + 0.006*"perform" + 0.006*"cbr" + 0.006*"apparently" + 0.005*"comfortable" + 0.005*"application" + 0.005*"everybody\'s" + 0.005*"4611"')
(2, u'0.016*"last" + 0.016*"search" + 0.012*"doctors" + 0.012*"submit" + 0.011*"easier" + 0.010*"folders" + 0.009*"confused" + 0.009*"tried" + 0.008*"elements" + 0.008*"matrix" + 0.008*"prior" + 0.007*"unique" + 0.006*"function" + 0.006*"quantile" + 0.006*"access" + 0.006*"iterating"

In [9]:
print type(a_docs[0])
for i in range(40):
    bow = dictionary.doc2bow(a_docs[i])
    t = a_ldamodel.get_document_topics(bow)
    print t

<type 'list'>
[(0, 0.12312915), (2, 0.33368683), (3, 0.19199309), (7, 0.14259973), (8, 0.18358934)]
[(0, 0.15770732), (1, 0.15363471), (2, 0.31442869), (3, 0.13680445), (7, 0.068488061), (9, 0.14893441)]
[(0, 0.20118041), (1, 0.1698007), (2, 0.24633662), (7, 0.15384947), (8, 0.09606643), (9, 0.11609899)]
[(0, 0.065525457), (1, 0.15235695), (2, 0.44759434), (7, 0.16310585), (8, 0.048088197), (9, 0.11563668)]
[(0, 0.14262626), (1, 0.15024574), (2, 0.37469843), (4, 0.058581553), (7, 0.13608751), (9, 0.11775806)]
[(0, 0.15665424), (2, 0.42993912), (3, 0.19113472), (4, 0.077682815), (6, 0.11827157)]
[(0, 0.16407874), (1, 0.15714538), (2, 0.57876742), (3, 0.014285714), (4, 0.014285714), (5, 0.014285714), (6, 0.014285714), (7, 0.014286349), (8, 0.014285935), (9, 0.014293291)]
[(1, 0.088629931), (2, 0.62462968), (3, 0.033957403), (4, 0.018670117), (7, 0.019739993), (8, 0.15925552), (9, 0.050769195)]
[(0, 0.02000102), (1, 0.020003537), (2, 0.23594579), (3, 0.33605123), (4, 0.020002719), (5, 0.0